In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, optimization
import os 
from datasets import load_dataset
import pandas as pd
import numpy as np 
from copy import deepcopy
from torch.optim import AdamW
import torch

C:\Users\Yoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [3]:
dream_checkpoint = "RicoBorra/DREAM-t5-small"

dream_tokenizer = AutoTokenizer.from_pretrained(dream_checkpoint)
dream_model = AutoModelForSeq2SeqLM.from_pretrained(dream_checkpoint)
dream_data_collator = DataCollatorForSeq2Seq(tokenizer=dream_tokenizer, model=dream_checkpoint)

## FLUTE Data extraction and processing
Using the few instructions from the Git Readme to have the same initial data

In [4]:
# Only the train split is available on HuggingFace
dataset = load_dataset("ColumbiaNLP/FLUTE")

In [40]:
def compute_dream_elaborations(dataset, dream_model, path_to_save) :
    if os.path.exists(path_to_save) :
        elaborations = pd.read_csv(path_to_save)
    else :
        elaborations = pd.DataFrame(columns=['premise_emotion', 'premise_motivation', 'premise_consequence', 'premise_rot',
                                            'hypothesis_emotion', 'hypothesis_motivation', 'hypothesis_consequence', 'hypothesis_rot'])
        for sentence_type in ['premise', 'hypothesis'] :
            for dream_dimension in ['emotion', 'motivation', 'consequence', 'rot'] :
                inputs = ['[SITUATION] ' + sentence + ' [QUERY] ' + dream_dimension for sentence in dataset[sentence_type]]
                tokens = torch.tensor(tokenizer(inputs, padding='longest').input_ids)
                output_tokens = dream_model.generate(tokens, max_new_tokens=100)
                elaborations[sentence_type + '_' + dream_dimension] = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
        
        # Make sure each sentence ends with a point
        elaborations = elaborations.applymap(lambda x : x + '.' if not x.endswith('.') else x)
        
        elaborations.to_csv(path_to_save)

    return elaborations

In [ ]:
dreams = compute_dream_elaborations(dataset['train'], dream_model, "dream_elaborations.csv")

def add_combined_cols(entry):
    
    premise = entry["premise"].strip()
    hypothesis = entry["hypothesis"].strip()
    
    if not premise.endswith("."):
        premise += "."
    assert(premise.endswith("."))
    if not hypothesis.endswith("."):
        hypothesis += "."
    assert(hypothesis.endswith("."))
    
    # Columns for System 1
    entry["premise_hypothesis"] = 'Premise: ' + premise + ' Hypothesis: ' + hypothesis + 'Is there a contradiction or entailment between the premise and hypothesis ?'
    entry["label_explanation"] = 'Label: ' + entry["label"] + '. Explanation: ' + entry["explanation"]

    # Columns for System 2
    entry["premise_hypothesis_system_2"] = 'Premise: ' + premise + ' Hypothesis: ' + hypothesis + 'What is the type of figurative language involved? Is there a contradiction or entailment between the premise and hypothesis ?'
    entry["type_label_explanation"] = 'Type: ' + entry["type"] + '. Label: ' + entry["label"] + '. Explanation: ' + entry["explanation"]
    
    # Columns for Systems 3
    for dream_dimension in ['emotion', 'motivation', 'consequence', 'rot'] :
        entry["premise_hypothesis_" + dream_dimension] = 'Premise: ' + premise + '[' + dream_dimension.capitalize() + ']' + dreams['premise_' + dream_dimension].strip() + \
                    ' Hypothesis: ' + hypothesis + '[' + dream_dimension.capitalize() + ']' + dreams['hypothesis_' + dream_dimension] + 'Is there a contradiction or entailment between the premise and hypothesis ?'
    entry["premise_hypothesis_all_dims"] = 'Premise: ' + premise + \
                '[Emotion]' + dreams['premise_emotion'].strip() + \
                '[Motivation]' + dreams['premise_motivation'].strip() + \
                '[Consequence]' + dreams['premise_consequence'].strip() + \
                '[Rot]' + dreams['premise_rot'].strip() + \
                ' Hypothesis: ' + hypothesis + \
                '[Emotion]' + dreams['hypothesis_emotion'].strip() + \
                '[Motivation]' + dreams['hypothesis_motivation'].strip() + \
                '[Consequence]' + dreams['hypothesis_consequence'].strip() + \
                '[Rot]' + dreams['hypothesis_rot'].strip()
    return entry
# combine columns
combined_cols_dataset = dataset['train'].map(add_combined_cols)

# create train test split because given data has only train data
# splits are shuffled by default
dataset_train_test = combined_cols_dataset.train_test_split(test_size=0.2, seed=42)

In [8]:
import evaluate
rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predicted_token_ids = torch.argmax(torch.from_numpy(predictions[0]), dim=-1)
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## System 1 : Normal classifier

In [9]:
model_s1 = deepcopy(model)

In [6]:
def preprocess_dataset(examples):
    model_inputs = tokenizer(examples['premise_hypothesis'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [7]:
tokenized_ds = dataset_train_test.map(preprocess_dataset, batched=True)
tokenized_ds = tokenized_ds.remove_columns(dataset_train_test['train'].column_names)

In [10]:
'''The following parameters were taken from the DREAM-FLUTE paper (only the number of epochs has been increased because the model is smaller)'''
training_args = Seq2SeqTrainingArguments(
    output_dir="D:\Documents\PoliTo\Deep NLP\Project\S1Model_more_accurate",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    seed=42,
    #weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,
    load_best_model_at_end=True,
    #eval_accumulation_steps=8,
    #fp16=True,
    #push_to_hub=True,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    lr_scheduler_type='linear'
)

trainer = Seq2SeqTrainer(
    model=model_s1,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics
)

trainer.train()

  2%|▏         | 501/24112 [01:00<44:48,  8.78it/s]

{'loss': 2.3937, 'learning_rate': 4.896317186463172e-05, 'epoch': 0.17}


  4%|▍         | 1001/24112 [02:02<47:53,  8.04it/s] 

{'loss': 2.0177, 'learning_rate': 4.792634372926344e-05, 'epoch': 0.33}


  6%|▌         | 1501/24112 [03:07<41:30,  9.08it/s]  

...

                                                     
100%|██████████| 24112/24112 [56:41<00:00,  7.79it/s]

{'eval_loss': 1.3940967321395874, 'eval_runtime': 30.05, 'eval_samples_per_second': 50.15, 'eval_steps_per_second': 25.092, 'epoch': 8.0}


100%|██████████| 24112/24112 [56:43<00:00,  7.08it/s]

{'train_runtime': 3403.4563, 'train_samples_per_second': 14.167, 'train_steps_per_second': 7.085, 'train_loss': 1.5525400064287393, 'epoch': 8.0}


TrainOutput(global_step=24112, training_loss=1.5525400064287393, metrics={'train_runtime': 3403.4563, 'train_samples_per_second': 14.167, 'train_steps_per_second': 7.085, 'train_loss': 1.5525400064287393, 'epoch': 8.0})

In [16]:
model_s1 = AutoModelForSeq2SeqLM.from_pretrained("D:\Documents\PoliTo\Deep NLP\Project\S1Model_more_accurate\checkpoint-24112")
i = "Premise: Today I crashed my car. Hypothesis: I felt like a champion when I crashed my car."
t = tokenizer(i, return_tensors='pt').input_ids
t = t.to(model_s1.device)
o = model_s1.generate(t, max_new_tokens = 100)
d = tokenizer.decode(o[0])
d 

'<pad> Label: Contradiction. Explanation: Accidentally driving a car is not a good thing and so someone feeling like a champion when they crash it is not a good thing.</s>'

## System 2 : Predict type of figurative language